In [1]:
import pde_control_gym 
import gymnasium as gym
import numpy as np
import math
import matplotlib.pyplot as plt
import time
# 加载一些工具
from utils import set_size
from utils import linestyle_tuple
from utils import load_csv
# use the stable_baselines3 
from stable_baselines3 import PPO
from stable_baselines3 import SAC
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import CheckpointCallback
# choose the pre-implemented reward function
from pde_control_gym.src import TunedReward1D

envs initialized


In [2]:
# 检查环境是否注册和依赖包
# # 打印所有注册环境的ID 检查所需要的环境是否导入
# for env_spec in gym.envs.registry.values():
#     print(env_spec.id)

# Print Versioning
# print("Gym version", gym.__version__)
# print("Numpy version", np.__version__)
# print("Stable Baselines3 version", stable_baselines3.__version__)

In [3]:
# NO NOISE
# lambda state : state 输入状态值返回状态值
def noiseFunc(state):
    return state

# Chebyshev Polynomial Beta Functions
def solveBetaFunction(x, gamma):
    # 先创建一个数组 shape =（len(x),）
    beta = np.zeros(len(x), dtype=np.float32)
    # 遍历赋值
    for idx, val in enumerate(x):
        # 在每一个离散点上计算beta
        beta[idx] = 5 * math.cos(gamma * math.acos(val))
    return beta

# Returns beta functions passed into PDE environment. Currently gamma is always
# set to 7.35, but this can be modified for further problems.
# This function is used to create the beta function for the PDE environment.
def getBetaFunction(nx):
    return solveBetaFunction(np.linspace(0, 1, nx), 7.35)

# Kernel function solver for backstepping
def solveKernelFunction(beta, dx):
    # theta 一个一维数组
    # 创建一个和 theta 一样长度的数组 kappa
    kappa = np.zeros(len(beta))
    # 索引从 0 到 len（theta）- 1，总的长度还是 len（theta）
    for i in range(0, len(beta)):
        kernelIntegral = 0
        # 矩形法离散积分函数值取左端点，所以只积分到 i-1 项
        for j in range(0, i):
            kernelIntegral += (kappa[i-j]*beta[j])*dx
        kappa[i] = kernelIntegral  - beta[i]
        # np.flip 用来翻转数组 （倒序 第一位为k（1））这样做是因为控制器是加权积分，（可以想象成卷积）因为状态是 u = [u(0), ····，u(1)]，对应的离散权重应该是 k =  [k(1), ····，u(0)]    
        # 则U(1,t) = np.sum(u * k) 按位相乘再相加
    return np.flip(kappa)

# Control convolution solver
def solveControl(kernel, u, dx):
    res = 0
    for i in range(len(u)):
        res += kernel[i]*u[i]
    return res*dx

# Set initial condition function here
def getInitialCondition(nx):
    # *是按位乘法
    return np.ones(nx)*np.random.uniform(1, 10)


In [4]:
# Timestep and spatial step for PDE Solver
T = 5
# 0.0001
dt = 1e-4
X = 1
# 0.01
dx = 1e-2

In [5]:
# 定义奖励函数：设置时间步、提前截断的单位时间步惩罚、正常终止的奖励
reward_class =  TunedReward1D(int(round(T/dt)), -1e3, 3e2)

# 先设置一个通用的参数字典
hyperbolicParameters = {
        "T": T, 
        "dt": dt, 
        "X": X,
        "dx": dx, 
        "reward_class": reward_class,
        "normalize":None, 
        "sensing_loc": "full", 
        "control_type": "Dirchilet", 
        "sensing_type": None,
        # 确定传感器返回的测量值是否添加噪声，这里的这个表示精确的返回状态，并未添加任何噪声；（*lambda*构建了一个简单的函数，输入state，返回state）
        "sensing_noise_func": lambda state: state,
        # 用于早期停止的参数
        "limit_pde_state_size": True,
        "max_state_value": 1e10,
        "max_control_value": 20,
        # 传入初始条件的函数
        "reset_init_condition_func": getInitialCondition,
        # 传入计算beta的函数
        "reset_recirculation_func": getBetaFunction,
        # 控制采样频率 数值仿真时需要很小的时间步长，但控制器的接收控制信号无法这么快
        "control_sample_rate": 0.1,
}

# All of the 1D PDE boundary control environments have the same set of optional parameters for ease of use! 

# 通过浅拷贝的方式设置 Backstepping 方法参数字典
hyperbolicParametersBackstepping = hyperbolicParameters.copy()
# 在复制过来的通用基础上修改某些关键参数 ⬇️ Normalize 专为强化学习控制器设计，如果设置为True，则控制器的动作值会被归一化到[-1, 1]之间,并根据"max_control_value"转换为实际控制值
hyperbolicParametersBackstepping["normalize"] = False

# 设置 Rl 参数字典
hyperbolicParametersRL = hyperbolicParameters.copy()
# 需要用到线性化
hyperbolicParametersRL["normalize"] = True

In [6]:
# These will allow us to compare two algorithms, namely PPO and SAC. From here, we can then train each algorithm on our environment for 500k timestep using the following code:

# Make environments
# This explitly is making two environments registered under the name PDEControlGym-HyperbolicPDE1D with two separate parameters dictionaries, namely hyperbolicParametersRL and hyperbolicParametersBackstepping （两个独立的参数字典）
# gym.make("环境名"（需要已经注册）， **参数字典)是Gym的标准调用格式
# # 创建强化学习控制器学习环境
# envRL = gym.make("PDEControlGym-TransportPDE1D", **hyperbolicParametersRL)
# # 创建Backstepping控制器环境
# envBcks = gym.make("PDEControlGym-TransportPDE1D",**hyperbolicParametersBackstepping)
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

envRL = DummyVecEnv([lambda: Monitor(gym.make("PDEControlGym-TransportPDE1D", **hyperbolicParametersRL))])
envRL = VecNormalize(envRL, norm_obs=True, norm_reward=True)

# 👀 创建一个训练参数实例 
checkpoint_callback = CheckpointCallback(
  # 每训练一万步保存一次模型，保证模型中途可以恢复
  save_freq=10000,
  # 保存路径 在当前目录下生成一个文件夹用于存放
  save_path="./logsPPO",
  # 保存文件名前缀，生成文件名为：rl_model_xxxx_step.zip
  name_prefix="rl_model",
  # 保存经验回放缓冲区（SAC需要）
  save_replay_buffer=True,
  # 保存环境标准化状态（避免训练与测试不一致）
  save_vecnormalize=True,
)


# Train PPO
# PPO模型参数设置，model 是 PPO 类的一个实例对象（对象里包含策略网络、优化器、训练参数等）
# MlpPolicy：使用多层感知机（全连接网络）作为策略和价值函数的近似器。
model = PPO("MlpPolicy",envRL, verbose=1, tensorboard_log="./tb/") # verbose 打印训练日志： 0 不输出；1 简单输出；2 详细输出  tensorboard_log 记录日志到 ./tb/目录，便于TensorBoard查看

# Train for 500k timesteps，并启用检查点保存机制
model.learn(total_timesteps=5e5, callback=checkpoint_callback) 

# # Train SAC
# model = SAC("MlpPolicy",envRL, verbose=1, tensorboard_log="./tb/")
# model.learn(total_timesteps=5e5, callback=checkpoint_callback)

Using cuda device
Logging to ./tb/PPO_1


/home/lab422/anaconda3/envs/TxlRL/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -358     |
| time/              |          |
|    fps             | 146      |
|    iterations      | 1        |
|    time_elapsed    | 13       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 50          |
|    ep_rew_mean          | -351        |
| time/                   |             |
|    fps                  | 143         |
|    iterations           | 2           |
|    time_elapsed         | 28          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.002469583 |
|    clip_fraction        | 0.0122      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.41       |
|    explained_variance   | -0.269      |
|    learning_rate        | 0.

In [ ]:
# Visualize Rewards

# In TensorBoard, save the avg rewards plot as a csv and then put their paths here
# Set your tensorboard avg_rew paths. WILL NEED UPDATING FOR USE
filenamesPPO = ["PPOData/test1.csv", "PPOData/test2.csv", "PPOData/test3.csv", "PPOData/test4.csv", "PPOData/test5.csv"]
filenamesSAC = ["SACData/SAC_18.csv", "SACData/SAC_19.csv", "SACData/SAC_20.csv", "SACData/SAC_21.csv", "SACData/SAC_23.csv"]

timePPOArr = []
rewardPPOArr = []
for name in filenamesPPO:
    times, rewards = load_csv(name)
    timePPOArr.append(times)
    rewardPPOArr.append(rewards)

timeSACArr = []
rewardSACArr = []
for name in filenamesSAC:
    times, rewards = load_csv(name)
    timeSACArr.append(times)
    rewardSACArr.append(rewards)

# takes max amount of timesteps all data has
maxTimestep = np.inf
for data in timePPOArr:
    maxTimestep = min(maxTimestep, data[-1])
for data in timeSACArr:
    maxTimestep = min(maxTimestep, data[-1])
print(maxTimestep)

# remove data after minTimestep
maxDataSeqPPO = []
for data in timePPOArr:
    for i in range(len(data)):
        if data[i] >= maxTimestep:
            maxDataSeqPPO.append(i)
            break
maxDataSeqSAC = []
for data in timeSACArr:
    for i in range(len(data)):
        if data[i] >= maxTimestep:
            maxDataSeqSAC.append(i)
            break

# Get mean and std of each value at time step 
rewardArrCleanPPO = []
for i, data in enumerate(rewardPPOArr):
    rewardArrCleanPPO.append(data[:min(maxDataSeqPPO)])
rewardArrPPO = np.array(rewardArrCleanPPO)
meanArrPPO = rewardArrPPO.mean(axis=0)
stdArrPPO = rewardArrPPO.std(axis=0)

rewardArrCleanSAC = []
for i, data in enumerate(rewardSACArr):
    rewardArrCleanSAC.append(data[:min(maxDataSeqSAC)])
rewardArrSAC = np.array(rewardArrCleanSAC)
meanArrSAC = rewardArrSAC.mean(axis=0)
stdArrSAC = rewardArrSAC.std(axis=0)

# Set size according to latex textwidth
fig = plt.figure(figsize=set_size(432, 0.99, (1, 1), height_add=0))
ax = fig.subplots(ncols=1)
t = timePPOArr[0]
x = t[:maxDataSeqPPO[0]]
mean = meanArrPPO
std = stdArrPPO
# 95 confidence interval
cis = (mean - 2*std, mean + 2*std)
ax.plot(x, mean, label="PPO")
ax.fill_between(x, cis[0], cis[1], alpha=0.2)

t = timeSACArr[0]
x = t[:min(maxDataSeqSAC)]
mean = meanArrSAC
std = stdArrSAC
# 95 confidence interval
cis = (mean - 2*std, mean + 2*std)
ax.plot(x, mean, label="SAC")
ax.fill_between(x, cis[0], cis[1], alpha=0.2)

plt.legend()
plt.title("Training Reward for Hyperbolic PDE")
plt.xlabel("Episode Number")
plt.ylabel("Average Reward")

In [ ]:
# Load RL models. # DUMMY ARGUMENTS NEED TO BE MODIFIED
ppoModelPath = "./hyperbolicModels/ppoBestModelHyperbolic"
sacModelPath = "./hyperbolicModels/sacBestModelHyperbolic"
ppoModel = PPO.load(ppoModelPath)
sacModel = SAC.load(sacModelPath)

# For backstepping controller
spatial = np.linspace(dx, X, int(round(X/dx)))
beta = solveBetaFunction(spatial, 7.35)

In [ ]:
# Runs a single epsiode calculation
# Parameter varies. For SAC and PPO it is the model itself
# For backstepping it is the beta function
def runSingleEpisode(model, env, parameter):
    terminate = False
    truncate = False

    # Holds the resulting states
    uStorage = []

    # Reset Environment
    obs,__ = env.reset()
    uStorage.append(obs)

    i = 0
    rew = 0
    while not truncate and not terminate:
        # use backstepping controller
        action = model(obs, parameter)
        obs, rewards, terminate, truncate, info = env.step(action)
        uStorage.append(obs)
        rew += rewards 
    u = np.array(uStorage)
    return rew, u

In [ ]:
# Define Controllers
def bcksController(obs, beta):
    kernel = solveKernelFunction(beta)
    return solveControl(kernel, obs)

def RLController(obs, model):
    action, _state = model.predict(obs)
    return action

def openLoopController(_, _a):
    return 0

In [ ]:
# Run comparisons
num_instances = 50
# Backstepping. Controller is slow so this will take some time.
total_bcks_reward = 0
for i in range(num_instances):
    rew, _ = runSingleEpisode(bcksController, envBcks, beta)
    total_bcks_reward += rew
print("Backstepping Reward Average:", total_bcks_reward/num_instances)

In [ ]:
# PPO
total_ppo_reward = 0
for i in range(num_instances):
    rew, _ = runSingleEpisode(RLController, envRL, ppoModel)
    total_ppo_reward += rew
print("PPO Reward Average:", total_ppo_reward/num_instances)

In [ ]:
# SAC
total_sac_reward = 0
for i in range(num_instances):
    rew, _ = runSingleEpisode(RLController, envRL, sacModel)
    total_sac_reward += rew
print("SAC Reward Average:", total_sac_reward/num_instances)

In [ ]:
# PLOT EXAMPLE PROBLEMS.

# First Build Same Initial Condition Environments
# Set initial condition function here
def getInitialConditionTen(nx):
    return np.ones(nx)*10

def getInitialConditionOne(nx):
    return np.ones(nx)*1

hyperbolicParametersBacksteppingTen = hyperbolicParametersBackstepping.copy()
hyperbolicParametersBacksteppingTen["reset_init_condition_func"] = getInitialConditionTen

hyperbolicParametersBacksteppingOne = hyperbolicParametersBackstepping.copy()
hyperbolicParametersBacksteppingOne["reset_init_condition_func"] = getInitialConditionOne

hyperbolicParametersRLTen = hyperbolicParametersRL.copy()
hyperbolicParametersRLTen["reset_init_condition_func"] = getInitialConditionTen

hyperbolicParametersRLOne = hyperbolicParametersRL.copy()
hyperbolicParametersRLOne["reset_init_condition_func"] = getInitialConditionOne

# Make environments
envBcksTen = gym.make("PDEControlGym-TransportPDE1D", **hyperbolicParametersBacksteppingTen)
envBcksOne = gym.make("PDEControlGym-TransportPDE1D", **hyperbolicParametersBacksteppingOne)

envRLTen = gym.make("PDEControlGym-TransportPDE1D", **hyperbolicParametersRLTen)
envRLOne = gym.make("PDEControlGym-TransportPDE1D", **hyperbolicParametersRLOne)

In [ ]:
rewBcksTen, uBcksTen = runSingleEpisode(bcksController, envBcksTen, beta)
rewBcksOne, uBcksOne = runSingleEpisode(bcksController, envBcksOne, beta)

rewPPOTen, uPPOTen = runSingleEpisode(RLController, envRLTen, ppoModel)
rewPPOOne, uPPOOne = runSingleEpisode(RLController, envRLOne, ppoModel)

rewSACTen, uSACTen = runSingleEpisode(RLController, envRLTen, sacModel)
rewSACOne, uSACOne = runSingleEpisode(RLController, envRLOne, sacModel)

rewOpenTen,uOpenTen = runSingleEpisode(openLoopController, envBcksTen, _)
rewOpenTen,uOpenOne = runSingleEpisode(openLoopController, envBcksOne, _)

In [ ]:
# PLOT OPENLOOOP EXAMPLE. PLOTS ARE NOT SCALED THE SAME ON Z SO MAY HAVE TO ADJUST
fig = plt.figure(figsize=set_size(433, 0.99, (1, 2), height_add=1))
subfigs = fig.subfigures(nrows=1, ncols=1, hspace=0)

subfig = subfigs
subfig.suptitle(r"Open-loop (U(t)=0) instability of transport PDE for u(x, 0)=1, 10")
subfig.subplots_adjust(left=0.03, bottom=0.05, right=1, top=0.95, wspace=0, hspace=0)

spatial = np.linspace(dx, X, int(round(X/dx)))
temporal = np.linspace(0, T, len(uPPOOne))
meshx, mesht = np.meshgrid(spatial, temporal)

ax = subfig.subplots(nrows=1, ncols=2, subplot_kw={"projection": "3d", "computed_zorder": False})

for axes in ax:
    for axis in [axes.xaxis, axes.yaxis, axes.zaxis]:
        axis._axinfo['axisline']['linewidth'] = 1
        axis._axinfo['axisline']['color'] = "b"
        axis._axinfo['grid']['linewidth'] = 0.2
        axis._axinfo['grid']['linestyle'] = "--"
        axis._axinfo['grid']['color'] = "#d1d1d1"
        axis.set_pane_color((1,1,1))

ax[0].view_init(10, 35)
ax[0].set_xlabel("x", labelpad=-3)
ax[1].set_xlabel("x", labelpad=-3)
ax[0].set_ylabel("Time", labelpad=-3)
ax[1].set_ylabel("Time", labelpad=-3)
ax[0].set_zlabel(r"$u(x, t)$", rotation=90, labelpad=-7)

ax[0].zaxis.set_rotate_label(False)
ax[0].set_xticks([0, 0.5, 1])
ax[0].tick_params(axis='x', which='major', pad=-3)
ax[1].tick_params(axis='x', which='major', pad=-3)
ax[0].tick_params(axis='y', which='major', pad=-3)
ax[1].tick_params(axis='y', which='major', pad=-3)
ax[0].tick_params(axis='z', which='major', pad=-1)
ax[1].tick_params(axis='z', which='major', pad=-1)

ax[0].plot_surface(meshx, mesht, uOpenOne, edgecolor="black",lw=0.2, rstride=50, cstride=2, 
                        alpha=1, color="white", shade=False, rasterized=True, antialiased=True)
test = np.ones(len(temporal))
vals = (uOpenOne.transpose())[-1] 
ax[0].plot(test[1:], temporal[1:], vals[1:], color="red", lw=0.1, antialiased=False, rasterized=False)
 
ax[1].plot_surface(meshx, mesht, uOpenTen, edgecolor="black",lw=0.2, rstride=50, cstride=2, 
                        alpha=1, color="white", shade=False, rasterized=True, antialiased=True)
ax[1].view_init(10, 35)
ax[1].zaxis.set_rotate_label(False)
ax[1].set_xticks([0, 0.5, 1])
test = np.ones(len(temporal))
vals = (uOpenTen.transpose())[-1] 
ax[1].plot(test[1:], temporal[1:], vals[1:], color="red", lw=0.1, antialiased=False, rasterized=False)

#plt.savefig("hyperbolicOpenloop.png", dpi=300)

In [ ]:
# PLOT EACH EXAMPLE. PLOTS ARE NOT SCALED THE SAME ON Z SO MAY HAVE TO ADJUST
fig = plt.figure(figsize=set_size(433, 0.99, (2, 3), height_add=1))
subfigs = fig.subfigures(nrows=2, ncols=1, hspace=0)

subfig = subfigs[0]
subfig.suptitle(r"Example trajectories for $u(0, x)=1$ with backstepping, PPO, and SAC")
subfig.subplots_adjust(left=0.03, bottom=0.05, right=1, top=0.95, wspace=0, hspace=0)
X = 1
dx = 1e-2
T = 5
spatial = np.linspace(dx, X, int(round(X/dx)))
temporal = np.linspace(0, T, len(uPPOOne))
meshx, mesht = np.meshgrid(spatial, temporal)

ax = subfig.subplots(nrows=1, ncols=3, subplot_kw={"projection": "3d", "computed_zorder": False})

for axes in ax:
    for axis in [axes.xaxis, axes.yaxis, axes.zaxis]:
        axis._axinfo['axisline']['linewidth'] = 1
        axis._axinfo['axisline']['color'] = "b"
        axis._axinfo['grid']['linewidth'] = 0.2
        axis._axinfo['grid']['linestyle'] = "--"
        axis._axinfo['grid']['color'] = "#d1d1d1"
        axis.set_pane_color((1,1,1))

ax[0].view_init(10, 35)
ax[0].set_xlabel("x", labelpad=-3)
ax[1].set_xlabel("x", labelpad=-3)
ax[2].set_xlabel("x", labelpad=-3)
ax[0].set_ylabel("Time", labelpad=-3)
ax[2].set_ylabel("Time", labelpad=-3)
ax[1].set_ylabel("Time", labelpad=-3)
ax[0].set_zlabel(r"$u(x, t)$", rotation=90, labelpad=-7)

ax[0].zaxis.set_rotate_label(False)
ax[0].set_xticks([0, 0.5, 1])
ax[0].tick_params(axis='x', which='major', pad=-3)
ax[1].tick_params(axis='x', which='major', pad=-3)
ax[2].tick_params(axis='x', which='major', pad=-3)
ax[0].tick_params(axis='y', which='major', pad=-3)
ax[1].tick_params(axis='y', which='major', pad=-3)
ax[2].tick_params(axis='y', which='major', pad=-3)
ax[0].tick_params(axis='z', which='major', pad=-1)
ax[1].tick_params(axis='z', which='major', pad=-1)
ax[2].tick_params(axis='z', which='major', pad=-1)

ax[0].plot_surface(meshx, mesht, uBcksOne, edgecolor="black",lw=0.2, rstride=50, cstride=1, 
                        alpha=1, color="white", shade=False, rasterized=True, antialiased=True)
test = np.ones(len(temporal))
vals = (uBcksOne.transpose())[-1] 
ax[0].plot(test[1:], temporal[1:], vals[1:], color="red", lw=0.1, antialiased=False, rasterized=False)
 
ax[1].plot_surface(meshx, mesht, uPPOOne, edgecolor="black",lw=0.2, rstride=50, cstride=1, 
                        alpha=1, color="white", shade=False, rasterized=True, antialiased=True)
ax[1].view_init(10, 35)
ax[1].zaxis.set_rotate_label(False)
ax[1].set_xticks([0, 0.5, 1])
test = np.ones(len(temporal))
vals = (uPPOOne.transpose())[-1] 
ax[1].plot(test[1:], temporal[1:], vals[1:], color="red", lw=0.1, antialiased=False, rasterized=False)
 
ax[2].plot_surface(meshx, mesht, uSACOne, edgecolor="black",lw=0.2, rstride=50, cstride=1, 
                        alpha=1, color="white", shade=False, rasterized=True, antialiased=True)
ax[2].view_init(10, 35)
ax[2].zaxis.set_rotate_label(False)
ax[2].set_xticks([0, 0.5, 1])
test = np.ones(len(temporal))
vals = (uSACOne.transpose())[-1] 
ax[2].plot(test[1:], temporal[1:], vals[1:], color="red", lw=0.1, antialiased=False, rasterized=False)

subfig = subfigs[1]
subfig.suptitle(r"Example trajectories for $u(0, x)=10$ with backstepping, PPO, and SAC")
subfig.subplots_adjust(left=0.03, bottom=0.05, right=1, top=0.95, wspace=0, hspace=0)
X = 1
dx = 1e-2
T = 5
spatial = np.linspace(dx, X, int(round(X/dx)))
temporal = np.linspace(0, T, len(uPPOOne))
meshx, mesht = np.meshgrid(spatial, temporal)

ax = subfig.subplots(nrows=1, ncols=3, subplot_kw={"projection": "3d", "computed_zorder": False})

for axes in ax:
    for axis in [axes.xaxis, axes.yaxis, axes.zaxis]:
        axis._axinfo['axisline']['linewidth'] = 1
        axis._axinfo['axisline']['color'] = "b"
        axis._axinfo['grid']['linewidth'] = 0.2
        axis._axinfo['grid']['linestyle'] = "--"
        axis._axinfo['grid']['color'] = "#d1d1d1"
        axis.set_pane_color((1,1,1))

ax[0].view_init(10, 35)
ax[0].set_xlabel("x", labelpad=-3)
ax[1].set_xlabel("x", labelpad=-3)
ax[2].set_xlabel("x", labelpad=-3)
ax[0].set_ylabel("Time", labelpad=-3)
ax[2].set_ylabel("Time", labelpad=-3)
ax[1].set_ylabel("Time", labelpad=-3)
ax[0].set_zlabel(r"$u(x, t)$", rotation=90, labelpad=-7)

ax[0].zaxis.set_rotate_label(False)
ax[0].set_xticks([0, 0.5, 1])
ax[0].tick_params(axis='x', which='major', pad=-3)
ax[1].tick_params(axis='x', which='major', pad=-3)
ax[2].tick_params(axis='x', which='major', pad=-3)
ax[0].tick_params(axis='y', which='major', pad=-3)
ax[1].tick_params(axis='y', which='major', pad=-3)
ax[2].tick_params(axis='y', which='major', pad=-3)
ax[0].tick_params(axis='z', which='major', pad=-1)
ax[1].tick_params(axis='z', which='major', pad=-1)
ax[2].tick_params(axis='z', which='major', pad=-1)

ax[0].plot_surface(meshx, mesht, uBcksTen, edgecolor="black",lw=0.2, rstride=50, cstride=1, 
                        alpha=1, color="white", shade=False, rasterized=True, antialiased=True)
test = np.ones(len(temporal))
vals = (uBcksTen.transpose())[-1] 
ax[0].plot(test[1:], temporal[1:], vals[1:], color="red", lw=0.1, antialiased=False, rasterized=False)
 
ax[1].plot_surface(meshx, mesht, uPPOTen, edgecolor="black",lw=0.2, rstride=50, cstride=1, 
                        alpha=1, color="white", shade=False, rasterized=True, antialiased=True)
ax[1].view_init(10, 35)
ax[1].zaxis.set_rotate_label(False)
ax[1].set_xticks([0, 0.5, 1])
test = np.ones(len(temporal))
vals = (uPPOTen.transpose())[-1] 
ax[1].plot(test[1:], temporal[1:], vals[1:], color="red", lw=0.1, antialiased=False, rasterized=False)
 
ax[2].plot_surface(meshx, mesht, uSACTen, edgecolor="black",lw=0.2, rstride=50, cstride=1, 
                        alpha=1, color="white", shade=False, rasterized=True, antialiased=True)
ax[2].view_init(10, 35)
ax[2].zaxis.set_rotate_label(False)
ax[2].set_xticks([0, 0.5, 1])
test = np.ones(len(temporal))
vals = (uSACTen.transpose())[-1] 
ax[2].plot(test[1:], temporal[1:], vals[1:], color="red", lw=0.1, antialiased=False, rasterized=False)

#plt.savefig("hyperbolicExamples.png", dpi=300)

In [ ]:
# BUILD CONTROL SIGNAL PLOTS 
fig = plt.figure(figsize=set_size(433, 0.99, (1, 2), height_add=1))
subfigs = fig.subfigures(nrows=1, ncols=1, hspace=0)

subfig = subfigs
subfig.suptitle(r"Control Signals for $u(0, x)=1$ and $u(0, x)=10$")
subfig.subplots_adjust(left=0.1, bottom=0.2, right=.98, top=0.86, wspace=0.25, hspace=0.1)
X = 1
dx = 1e-2
T = 10
spatial = np.linspace(dx, X, int(round(X/dx)))
temporal = np.linspace(0, T, len(uPPOOne))
ax = subfig.subplots(nrows=1, ncols=2)
l2, = ax[0].plot(temporal, uSACOne.transpose()[-1], label="SAC", linestyle=linestyle_tuple[2][1], color="green")
l1, = ax[0].plot(temporal, uPPOOne.transpose()[-1], label="PPO", linestyle=linestyle_tuple[2][1], color="orange")
l3, = ax[0].plot(temporal, uBcksOne.transpose()[-1], label="Backstepping", color="#0096FF")
ax[0].set_xlabel("Time")
ax[0].set_ylabel(R"$U(t)$", labelpad=-2)

l2, = ax[1].plot(temporal, uSACTen.transpose()[-1], label="SAC", linestyle=linestyle_tuple[2][1], color="green")
l1, = ax[1].plot(temporal, uPPOTen.transpose()[-1], label="PPO", linestyle=linestyle_tuple[2][1], color="orange")
l3, = ax[1].plot(temporal, uBcksTen.transpose()[-1], label="Backstepping", color="#0096FF")
ax[1].set_xlabel("Time")
ax[1].set_ylabel(r"$U(t)$", labelpad=-2)
plt.legend([l1, l2, l3], ["PPO", "SAC", "Backstepping"], loc="lower left", bbox_to_anchor=[.56,.86], reverse=True)
plt.legend(handletextpad=0.1)

#plt.savefig("hyperbolicControlSignals.png", dpi=300)

In [ ]:
# PDE L2 Error
def getPDEl2(u, uhat):
    nt = len(u)
    nx = len(u[0])
    pdeError = np.zeros(nt-1)
    for i in range(1, nt):
        error = 0
        for j in range(nx):
            error += (u[i][j] - uhat[i][j])**2
        error = np.sqrt(error)
        pdeError[i-1] = error
    return pdeError

In [ ]:
# Share Rewards and L2 Norms for each problem
print(("InitialCondition\tModel Trained\tHyperbolic1D Rewards\tHyperbolic1DTotalL2Norm").expandtabs(30))
print(("u(x, 0)=1\tBackstepping\t" + str(rewBcksOne) +"\t" + str(sum(getPDEl2(uBcksOne, np.zeros(uBcksOne.shape))))).expandtabs(30))
print(("u(x, 0)=1\tPPO\t" + str(rewPPOOne) +"\t" + str(sum(getPDEl2(uPPOOne, np.zeros(uBcksOne.shape))))).expandtabs(30))
print(("u(x, 0)=1\tSAC\t" + str(rewSACOne) +"\t" + str(sum(getPDEl2(uSACOne, np.zeros(uBcksOne.shape))))).expandtabs(30))
print(("u(x, 0)=10\tBackstepping\t" + str(rewBcksTen) +"\t" + str(sum(getPDEl2(uBcksTen, np.zeros(uBcksOne.shape))))).expandtabs(30))
print(("u(x, 0)=10\tPPO\t" + str(rewPPOTen) +"\t" + str(sum(getPDEl2(uPPOTen, np.zeros(uBcksOne.shape))))).expandtabs(30))
print(("u(x, 0)=10\tSAC\t" + str(rewSACTen) +"\t" + str(sum(getPDEl2(uSACTen, np.zeros(uBcksOne.shape))))).expandtabs(30))